In [ ]:
from time import sleep
from datetime import datetime
from tradingview_screener import Query, col
# Set display options to prevent column wrapping
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0) # Adjust as needed, 0 will use the full available width

from colorama import Fore, Back, Style     
import urllib.parse
 


from collections import defaultdict
from tvDatafeed import TvDatafeed,Interval
tv = TvDatafeed()
support_line = defaultdict(float)
resistance_line = defaultdict(float)
import csv

# def add_string_to_csv(filename, symbol):
#     """
#     Adds a string to a CSV file, writing each character as a separate field in a new row.

#     Args:
#         filename (str): The name of the CSV file to write to.
#         string_data (str): The string to add to the CSV file.
#     """
#     with open(filename, 'a', newline='\n') as csvfile:
#         current_time = datetime.now().strftime('%H:%M:%S')
#         writer = csv.writer(csvfile)
#         # strData = f":Alert for {symbol} ::  'https://in.tradingview.com/chart/N8zfIJVK/?symbol={symbol}' : Support Line: {support_line[symbol]}, Resistance Line: {resistance_line[symbol]}"
#         writer.writerow([current_time,symbol,f'https://in.tradingview.com/chart/N8zfIJVK/?symbol={symbol}',])

# def write_to_csv(symbol, minute_data,length): 
#     add_string_to_csv('SCREENER ALERTS_HIGH_VALUE'+str(current_DATE)+'.csv',symbol)
#     return True
   


master_combined =  pd.DataFrame()

import pandas as pd
import os

def append_df_to_csv(df, csv_path):
    if not os.path.exists(csv_path):
        df.to_csv(csv_path, mode='a', header=True, index=False)
    else:
        df.to_csv(csv_path, mode='a', header=False, index=False)


HIGH_VALUE = 20000000.00
VOLUME_MULTIPLIER =  5.0
writeHeader=True
while True : 
  
 
    # tvQuery = Query().set_markets('india') 
    # _,dfNew = (tvQuery
    #  .select('name', 'close', 'volume', 'volume|1','Value.Traded|1','average_volume_10d_calc','average_volume_10d_calc|1')
    # .where(        
    #         col('Value.Traded|1')> HIGH_VALUE 
    #     )
    # # .order_by('relative_volume_intraday|5', ascending=False)
    # .order_by('volume|1', ascending=False).get_scanner_data()) 

    #     query = Query().set_markets('india')
        
        # Select the columns we want to display
    query = Query().select(
        'name',                    # Stock name
        'close',                   # Current price
        'volume|1',                # 1-minute volume
        'Value.Traded|1',          # 1-minute traded value
        'average_volume_10d_calc', # Average volume (10 days)
        'average_volume_10d_calc|1' ,# Average 1-minute volume (10 days)
        'average_volume'
    ).where(
        col('is_primary') == True,
        col('typespecs').has('common'),
        col('type') == 'stock',
        col('exchange') == 'NSE',
        col('close').between(2, 10000),
        col('active_symbol') == True,
        col('Value.Traded|1') > HIGH_VALUE,

    ).order_by(
        'volume|1', ascending=False
    ).limit(100).set_markets('india').set_property(
        'preset', 'all_stocks'
    ).set_property(
        'symbols', {'query': {'types': ['stock', 'fund', 'dr']}}
    )
    # Execute the query
    _, dfNew = query.get_scanner_data()
   
    if not dfNew.empty :
        current_time = datetime.now().strftime('%H:%M:%S')
        
        current_DATE = datetime.now().strftime('%d_%m_%y')

        # dfNew.insert(2, 'baseURL', "https://in.tradingview.com/chart/N8zfIJVK/?symbol="+str(dfNew['ticker'])+"") 
        # Insert the current time as a new column at the beginning of the DataFrame
        dfNew.insert(3, 'URL',"")  
        dfNew['encodedTicker'] = dfNew['ticker'].apply(urllib.parse.quote)
        dfNew['URL'] = "https://in.tradingview.com/chart/N8zfIJVK/?symbol="+dfNew['encodedTicker']+" "

        dfNew.insert(0, 'current_timestamp', current_time)
        # Filter rows where col1 is 3 times col2
        # 
        dfNew['VOL_MULT_FACTOR'] = dfNew['volume|1'] / dfNew['average_volume_10d_calc|1']
        filtered_df =   dfNew[dfNew['VOL_MULT_FACTOR'] > VOLUME_MULTIPLIER] 
        
        if not filtered_df.empty : 
            append_df_to_csv(filtered_df, 'HIGH_VALUE_VolSpike_'+str(current_DATE)+'.csv')
            print(f"=="*30)
            # print(filtered_df)
            writeHeader=False 
            print(filtered_df)
        
               
                
    sleep(30)
 

you are using nologin method, data you access may be limited


  current_timestamp       ticker     name   close                                                URL  volume|1  Value.Traded|1  average_volume_10d_calc  average_volume_10d_calc|1  average_volume  encodedTicker  VOL_MULT_FACTOR
0          13:33:18  NSE:TFCILTD  TFCILTD  309.85  https://in.tradingview.com/chart/N8zfIJVK/?sym...    142929     44286550.65                1536860.9                    14620.2       1516590.1  NSE%3ATFCILTD         9.776132
  current_timestamp       ticker     name   close                                                URL  volume|1  Value.Traded|1  average_volume_10d_calc  average_volume_10d_calc|1  average_volume  encodedTicker  VOL_MULT_FACTOR
0          13:33:49     NSE:IDEA     IDEA    6.82  https://in.tradingview.com/chart/N8zfIJVK/?sym...  13043160     88693488.00             4.474513e+08                  2044820.5    4.404649e+08     NSE%3AIDEA         6.378633
1          13:33:49  NSE:TFCILTD  TFCILTD  309.85  https://in.tradingview.com/chart/N8zfIJVK